# WP4: Climatology and Bias - Near Surface Air Temperature

## Import libraries

Switch warnings off for better readability.\
Set matplotlib default settings.

In [ ]:
import warnings

import matplotlib.pyplot as plt
import xarray as xr
from c3s_eqc_automatic_quality_control import diagnostics, download, plot

plt.style.use("seaborn-v0_8-talk")

warnings.filterwarnings("ignore")

## Define time period

In [ ]:
year_start = 1985
year_stop = 2014

common_request = {
    "year": [str(year) for year in range(year_start, year_stop + 1)],
    "month": [f"{month:02d}" for month in range(1, 12 + 1)],
}

## Define requests

In [ ]:
request_era = (
    "reanalysis-era5-single-levels-monthly-means",
    {
        "product_type": "monthly_averaged_reanalysis",
        "format": "netcdf",
        "time": "00:00",
        "variable": "2m_temperature",
        **common_request,
    },
)

request_sim = (
    "projections-cmip6",
    {
        "format": "zip",
        "temporal_resolution": "monthly",
        "experiment": "historical",
        "variable": "near_surface_air_temperature",
        "model": "cmcc_cm2_sr5",
        **common_request,
    },
)

## Download data and interpolate

In [ ]:
ds_era = download.download_and_transform(
    *request_era,
    chunks={"year": 1},
)

ds_sim = download.download_and_transform(
    *request_sim,
    transform_func=diagnostics.regrid,
    transform_func_kwargs={
        "grid_out": ds_era[["longitude", "latitude"]],
        "method": "bilinear",
        "periodic": True,
    },
    chunks={"year": 1},
)

## Postprocessing

In [ ]:
# Compute seasonality and bias
ds_era_seasonal = diagnostics.seasonal_weighted_mean(ds_era)
ds_sim_seasonal = diagnostics.seasonal_weighted_mean(ds_sim)


# Convert kelvin to celsius
def kelvin_to_celsius(da):
    """Convert from kelvin to celsius."""
    with xr.set_options(keep_attrs=True):
        da -= 273.15
    da.attrs["units"] = "°C"
    return da


da_era = kelvin_to_celsius(ds_era_seasonal["t2m"])
da_sim = kelvin_to_celsius(ds_sim_seasonal["tas"])

# Compute bias
with xr.set_options(keep_attrs=True):
    da_bias = da_sim - da_era
da_bias.attrs["long_name"] += " Bias"

## Define plot settings

In [ ]:
# Plot settings
sim_kwargs = {"levels": range(-30, 31, 5), "cmap": "YlOrRd"}
bias_kwargs = {"levels": range(-6, 7), "cmap": "RdBu_r"}
facet_kwargs = {"col": "season", "col_wrap": 2}
savefig_kwargs = {"bbox_inches": "tight"}
suptitle_kwargs = {"x": 0.5, "y": 1, "ha": "center", "va": "bottom"}

# Labels
info = {key: request_sim[1][key] for key in ("model", "experiment", "variable")}
info["period"] = f"{request_sim[1]['year'][0]}-{request_sim[1]['year'][-1]}"
fig_prefix = "_".join(list(info.values()))
suptitle = "; ".join([f"{k.title()}: {v}" for k, v in info.items()])

## Plots

In [ ]:
# DJF
p1 = plot.global_map(da_sim.sel(season=["DJF"]), **sim_kwargs)
p1.figure.suptitle(suptitle, **suptitle_kwargs)
p1.figure.savefig(f"{fig_prefix}_DJF.png", **savefig_kwargs)

In [ ]:
# DJF bias
p2 = plot.global_map(da_bias.sel(season=["DJF"]), **bias_kwargs)
p2.figure.suptitle(suptitle, **suptitle_kwargs)
p2.figure.savefig(f"{fig_prefix}_bias_DJF.png", **savefig_kwargs)

In [ ]:
# All seasons
p3 = plot.global_map(da_sim, **sim_kwargs, **facet_kwargs)
p3.fig.suptitle(suptitle, **suptitle_kwargs)
p3.fig.savefig(f"{fig_prefix}_seasonal.png", **savefig_kwargs)

In [ ]:
# All seasons bias
p4 = plot.global_map(da_bias, **bias_kwargs, **facet_kwargs)
p4.fig.suptitle(suptitle, **suptitle_kwargs)
p4.fig.savefig(f"{fig_prefix}_seasonal_bias.png", **savefig_kwargs)